In [ ]:
!pip install tokenizers
!pip install transformers

In [ ]:
import pandas as pd 
from torch.utils.data import Dataset
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased', output_hidden_states=True).eval()

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def find_cosin_similarity(text_1, text_2, text_3):
    tok1 = tokenizer(text_1, return_tensors='pt')
    tok2 = tokenizer(text_2, return_tensors='pt')
    tok3 = tokenizer(text_3, return_tensors='pt')

    with torch.no_grad():
        out1 = model(tok1.input_ids)
        out2 = model(tok2.input_ids)
        out3 = model(tok3.input_ids)
    
    # Only grab the last hidden state
    states1 = out1.last_hidden_state.squeeze()
    states2 = out2.last_hidden_state.squeeze()
    states3 = out3.last_hidden_state.squeeze()
    
    # average words vectors
    avg1 = states1.mean(axis=0)
    avg2 = states2.mean(axis=0)
    avg3 = states3.mean(axis=0)
    
    cosin_origin = torch.cosine_similarity(avg1.reshape(1,-1), avg2.reshape(1,-1))
    cosin_non_toxic = torch.cosine_similarity(avg1.reshape(1,-1), avg3.reshape(1,-1))
    if cosin_origin < cosin_non_toxic: 
        who_won = 'toxic_original'
    else: 
        who_won = 'non_toxic_ipa'

    return cosin_origin, cosin_non_toxic, who_won

In [ ]:
data = pd.read_csv('/content/dataset_to_model.csv')
replica_1 = list(data["Реплика 1 toxicity"])
replica_2_toxic_original = list(data["Реплика 2 toxicity"])
replica_2_nontoxic_ipa_1 = list(data["Реплика 2 original dialogue 1"])
#replica_2_nontoxic_ipa_2 = list(data["Реплика 2 original dialogue 2"])


In [ ]:
cosins_origins_toxic = []
cosins_non_toxics_ipa = []
who_wins = []
for text_1, text_2, text_3 in tqdm(zip(replica_1, replica_2_toxic_original, replica_2_nontoxic_ipa_1)):
    cosin_origin, cosin_non_toxic, who_won = find_cosin_similarity(text_1, text_2, text_3)
    cosins_origins_toxic.append(cosin_origin)
    cosins_non_toxics_ipa.append(cosin_non_toxic)
    who_wins.append(who_won)

data.cosins_origins = cosins_origins_toxic 
data.cosins_non_toxics_1 = cosins_non_toxics_ipa
data.who_wins_1 = who_wins

13321it [1:56:19,  1.91it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  if sys.path[0] == '':


In [ ]:
data['cosins_non_toxics_1'] = cosins_non_toxics_ipa
data['who_wins_1'] = who_wins

In [ ]:
data['cosins_origins'] = cosins_origins_toxic 

In [ ]:
data.to_csv('rubert.csv')

In [ ]:
cosins_origins_toxic = []
cosins_non_toxics_ipa = []
who_wins = []
for text_1, text_2, text_3 in tqdm(zip(replica_1, replica_2_toxic_original, replica_2_nontoxic_ipa_2)):
    cosin_origin, cosin_non_toxic, who_won = find_cosin_similarity(text_1, text_2, text_3)
    cosins_origins_toxic.append(cosin_origin)
    cosins_non_toxics_ipa.append(cosin_non_toxic)
    who_wins.append(who_won)

data.cosins_non_toxics_2 = cosins_non_toxics_ipa
data.who_wins_2 = who_wins

13321it [1:55:58,  1.91it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
data.to_csv('rubert_2.csv')

In [1]:
import pandas as pd

data = pd.read_csv('rubert.csv')
data['who_wins_1'].value_counts()

toxic_original    12406
non_toxic_ipa       915
Name: who_wins_1, dtype: int64

In [2]:
data.head()

,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group,ipm,zipf,Реплика 2 original dialogue 1,Реплика 2 original dialogue 2,cosins_non_toxics_1,who_wins_1,cosins_origins
0,0,м когда мужики спокойно за прыгают. пиздомраз...,слышишь блядь заткни свой ебальник сука,True,True,0.990932,0.997031,150 м Когда мужики спокойно за 250 прыгают. П...,Слышишь блядь заткни свой ебальник сука,"['слышать', 'блядь', 'заткнуть', 'свой', 'ебал...","блядь, сука",women,"[3642.207530790995, 1712.2843529757424]","[1507.0, 430.57142857142856]",Это ты сейчас о чём?,Вот они и хотят быть равноправными участниками...,tensor([0.4488]),toxic_original,tensor([0.4709])
1,1,журналюхи развели проф. бабуина на разговор о ...,я высказал предположение что ... но естествен...,True,True,0.983081,0.994750,журналюхи развели проф. Бабуина на разговор о ...,я высказал предположение что ... но естествен...,"['я', 'высказать', 'предположение', 'что', '.....","блядь, сука",women,"[3642.207530790995, 1712.2843529757424]","[1507.0, 430.57142857142856]",Такое случается и в жизни 🙁Лучше поменьше чита...,"Мне кажется, тут неуместно говорить «педераст»...",tensor([0.4352]),toxic_original,tensor([0.7187])
2,2,возможности карьеры прям про двачера для двач...,блядь сука пиздец вот повезло ебаный рот за чт...,False,True,0.079426,0.996924,Возможности карьеры 1991 Прям про двачера для...,БЛЯДЬ СУКА ПИЗДЕЦ ВОТ ПОВЕЗЛО ЕБАНЫЙ РОТ ЗА ЧТ...,"['блядь', 'сук', 'пиздец', 'вот', 'повезти', '...","блядь, сука",women,"[3642.207530790995, 1712.2843529757424]","[1507.0, 430.57142857142856]",А что случилось с твоими возможностями в 1991 ...,А сейчас у тебя какая карьера?,tensor([0.3511]),toxic_original,tensor([0.4075])
3,3,"в византии не было университетов, мань",в византии были государственные школы. до юсти...,False,True,0.001312,0.975552,"В Византии не было университетов, мань",В Византии были государственные школы. До Юсти...,"['в', 'византия', 'быть', 'государственный', '...","блядь, сука",women,"[3642.207530790995, 1712.2843529757424]","[1507.0, 430.57142857142856]",У них были только академии,Как же так? Там были философские школы,tensor([0.4177]),toxic_original,tensor([0.5465])
4,4,я думал на земле нет таких людей которые смогл...,лол мы чуть - не проебали сука я убил куры уби...,False,True,0.001034,0.995478,я думал на земле нет таких людей которые смогл...,ЛОЛ мы чуть 4-5 не проебали СУКА Я УБИЛ КУРЫ У...,"['лола', 'мы', 'чуть', '-', 'не', 'проебали', ...","блядь, сука",women,"[3642.207530790995, 1712.2843529757424]","[1507.0, 430.57142857142856]","Согласна, есть много странных и даже опасных в...","Согласна. Я слышала, что даже среди вегетариан...",tensor([0.6395]),non_toxic_ipa,tensor([0.4455])


In [ ]:
# cosins_origins = []
# cosins_toxics = []
# who_wins = []
# for text_1, text_2, text_3 in tqdm(zip(dialog_1, dialog_2_origin, dialog_2_toxic)):
#     cosin_origin, cosin_toxic, who_won = find_cosin_similarity(text_1, text_2, text_3)
#     cosins_origins.append(cosin_origin)
#     cosins_toxics.append(cosin_toxic)
#     who_wins.append(who_won)

# df.cosins_origins = cosins_origins
# df.cosins_toxics = cosins_toxics
# df.who_wins = who_wins